In [1]:
# import pandas as pd
# import numpy as np
# import nltk
# import os
# import json
# import requests 
# import time
# import re
# # import env_miatta as e
# # from wrangle import tokenize,clean,nlp_wrangle,intersection_list,extra_clean_column as w
# import matplotlib.pyplot as plt
# import unicodedata
# from bs4 import BeautifulSoup
# from typing import Dict, List, Optional, Union, cast
# from env_miatta import github_token, github_username
# from pprint import pprint
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# from sklearn.metrics import accuracy_score
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.ensemble import RandomForestClassifier
# from nltk.tokenize.toktok import ToktokTokenizer
# from nltk.corpus import stopwords
# %matplotlib inline
# # from time import strftime
# #from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# import pandas as pd
# import numpy as np
# import time
# import os
# import json
# import wrangle as w
# import model as m

# # Data visualization and manipulation
# import matplotlib.pyplot as plt
# import wordcloud 
# import seaborn as sns
# from pprint import pprint

# # Natural language processing and modeling
# import nltk.sentiment
# import nltk
# import re
# from scipy.stats import f_oneway, stats
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
# from nltk.tokenize import ToktokTokenizer
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import CountVectorizer

# # SQL credentials and data acquisition
# # import env as e
# # import acquire as a
# from typing import Dict, List, Optional, Union, cast
# import requests
# from bs4 import BeautifulSoup

# # GitHub API credentials
# # from env import github_token, github_username

# import warnings

# Import Libraries
import os
import time
import json

# Data Manipulation and Analysis
import pandas as pd
import numpy as np

# Custom Modules
import wrangle as w
import model as m

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import wordcloud 

# Natural Language Processing and Modeling
import nltk.sentiment
import nltk
from nltk.tokenize import ToktokTokenizer
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import f_oneway, stats
from typing import Dict, List, Optional, Union, cast

# Web Scraping
import requests
from bs4 import BeautifulSoup
from env_miatta import github_token, github_username as e

# Warnings
import warnings



# ACQUIRE

DATA TYPE SUMMARY

26 integer data types originally now 3
9 object data type originally now 1
0 null values
no missing values

In [2]:
# Aquire data using methods described above.
#Read the combined JSON data into a DataFrame
words_df = w.pd.read_json('data2.json')
words_df



,repo,language,readme_contents
0,TheAlgorithms/Python,Python,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
1,apache/flink,Java,# Apache Flink\n\nApache Flink is an open sour...
2,forezp/SpringCloudLearning,Java,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...
3,learn-co-students/python-dictionaries-readme-d...,Jupyter Notebook,\n# Dictionaries \n\n### Introduction\n\nAfter...
4,angular/angular-phonecat,JavaScript,# AngularJS Phone Catalog Tutorial Application...
...,...,...,...
113,learn-co-students/javascript-arrays-lab-bootca...,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
114,tastejs/todomvc,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
115,learn-co-students/jupyter-notebook-introductio...,Jupyter Notebook,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
116,hasura-imad/imad-2016-app,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."


In [3]:
# Dataset columns
words_df.columns

Index(['repo', 'language', 'readme_contents'], dtype='object')

In [4]:
words_df.language.value_counts()

JavaScript          31
Python              15
Java                11
HTML                 7
C                    6
Jupyter Notebook     5
Shell                5
Ruby                 4
C++                  3
CSS                  3
Go                   3
TypeScript           3
SCSS                 2
PowerShell           1
Kotlin               1
Bicep                1
HCL                  1
Vim script           1
Objective-C          1
Vue                  1
Name: language, dtype: int64

In [5]:
# To inspect the first few rows of the DataFrame, you can use the head functiotelco.head()
words_df.head().T


,0,1,2,3,4
repo,TheAlgorithms/Python,apache/flink,forezp/SpringCloudLearning,learn-co-students/python-dictionaries-readme-d...,angular/angular-phonecat
language,Python,Java,Java,Jupyter Notebook,JavaScript
readme_contents,"<div align=""center"">\n<!-- Title: -->\n <a hr...",# Apache Flink\n\nApache Flink is an open sour...,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...,\n# Dictionaries \n\n### Introduction\n\nAfter...,# AngularJS Phone Catalog Tutorial Application...


# PREPARE

### Data Cleaning:
    
- Drop unnecessary axis
- Rename
- Find nulls
- Drop nulls
- Check preperation
- The data set has 4 columns and 1,470 rows
- Each row represents individual employee numerical data
- Each column is attributes of the employees

In [6]:
# Check for total of duplicates in data set 
words_df.duplicated().sum()

0

In [7]:
# column datatypes and missign values
words_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             118 non-null    object
 1   language         105 non-null    object
 2   readme_contents  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [8]:
# observed numerical values
words_df.describe()


,repo,language,readme_contents
count,118,105,118
unique,118,20,118
top,TheAlgorithms/Python,JavaScript,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
freq,1,31,1


In [9]:
words_df.shape


(118, 3)

In [10]:
# Check for missing values
missing_values = words_df.isnull()
missing_values


,repo,language,readme_contents
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
113,False,False,False
114,False,False,False
115,False,False,False
116,False,False,False


In [11]:
words_df.isnull().sum()


repo                0
language           13
readme_contents     0
dtype: int64

In [12]:
# Drop rows with any missing values
words_df_dropped = words_df.dropna()
words_df_dropped


,repo,language,readme_contents
0,TheAlgorithms/Python,Python,"<div align=""center"">\n<!-- Title: -->\n <a hr..."
1,apache/flink,Java,# Apache Flink\n\nApache Flink is an open sour...
2,forezp/SpringCloudLearning,Java,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...
3,learn-co-students/python-dictionaries-readme-d...,Jupyter Notebook,\n# Dictionaries \n\n### Introduction\n\nAfter...
4,angular/angular-phonecat,JavaScript,# AngularJS Phone Catalog Tutorial Application...
...,...,...,...
113,learn-co-students/javascript-arrays-lab-bootca...,JavaScript,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
114,tastejs/todomvc,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
115,learn-co-students/jupyter-notebook-introductio...,Jupyter Notebook,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
116,hasura-imad/imad-2016-app,JavaScript,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."


In [13]:
rows_with_missing_language = words_df[words_df['language'].isnull()]

# Change data types of the isolated rows to 'object'
rows_with_missing_language = rows_with_missing_language.astype({'language': 'object'})

# Display the modified isolated rows
rows_with_missing_language


,repo,language,readme_contents
9,vivienzou1/DL-Notes-for-interview,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
11,jtleek/datasharing,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
13,freefq/free,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
17,996icu/996.ICU,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
26,luchihoratiu/debug-via-ssh,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
29,mqyqingfeng/Blog,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
40,xitu/gold-miner,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
41,xingshaocheng/architect-awesome,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."
54,dcxy/learngit,None,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" d..."
68,bloominstituteoftechnology/module-challenge-in...,None,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."


In [14]:
# column datatypes 
words_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             118 non-null    object
 1   language         105 non-null    object
 2   readme_contents  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [15]:
# Replace null values in the 'language' column with "Other"
words_df['language'].fillna("Other", inplace=True)

In [16]:
#column datatypes 
words_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             118 non-null    object
 1   language         118 non-null    object
 2   readme_contents  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [17]:
words_df = w.nlp_wrangle()
intersect =w.intersection_list()
words_df =w.extra_clean_column(words_df,intersect)
words_df.head().T

,0,1,2,3,4
repo,TheAlgorithms/Python,apache/flink,forezp/SpringCloudLearning,learn-co-students/python-dictionaries-readme-d...,angular/angular-phonecat
language,Python,Java,Java,Other,JavaScript
readme_contents,"<div align=""center"">\n<!-- Title: -->\n <a hr...",# Apache Flink\n\nApache Flink is an open sour...,>转载请标明出处： \n> http://blog.csdn.net/forezp/arti...,\n# Dictionaries \n\n### Introduction\n\nAfter...,# AngularJS Phone Catalog Tutorial Application...
clean_contents,gn center title href thelrithms src rw u...,che fnk che fnk oen source strem rocessing frm...,blog csdn net forez rticle detil blog csdn ...,dictionry introduction introducing working st ...,ngrjs hone ctlog tutoril ction overview ction ...
message_length,1585,2722,4370,5385,6259
word_count,257,454,617,828,1028
extra_clean_contents,b better blob blue build center code contribut...,bug build building built che check clone code ...,boot boots bus center cloud cor discovery f fe...,bee beginning bit built ce continue correct co...,b best binding bine building cent check checko...


### TRAIN SPLIT

In [19]:
# Split the dataset into training, validation, and test sets based on the 'language' variable
train, validate, test = m.split_data(words_df, 'language')

# Display the first few rows of the training dataset
train.head()


NameError: name 'train_test_split' is not defined

# EXPLORE

In [ ]:
#use the graph you havw=e in wrngle

In [ ]:
# Visual of dataset after cleaning and split 

words_df.hist(figsize=(10,10))
plt.show()

In [ ]:
# Call the create_bar_chart function to visualize the distribution of categories
w.create_bar_chart(words_df, 'language', 'Distribution of Categories')


In [ ]:
# Extract words from READMEs written in Python language and clean them
python_words = clean(' '.join(train[train.language=="Python"]['extra_clean_contents']))

# Extract words from READMEs written in Java language and clean them
java_words = clean(' '.join(train[train.language=="Java"]['extra_clean_contents']))

# Extract words from READMEs written in JavaScript language and clean them
script_words = clean(' '.join(train[train.language=="JavaScript"]['extra_clean_contents']))

# Extract words from READMEs written in HTML language and clean them
html_words = clean(' '.join(train[train.language=="HTML"]['extra_clean_contents']))

# Extract words from READMEs written in TypeScript language and clean them
type_words = clean(' '.join(train[train.language=="TypeScript"]['extra_clean_contents']))

# Extract words from READMEs not categorized into specific languages and clean them
other_words = clean(' '.join(train[train.language=="Other"]['extra_clean_contents']))

# Extract words from READMEs in the test set and clean them
all_words = clean(' '.join(test['extra_clean_contents']))


In [ ]:
# Calculate the word frequency for Python language
python_freq = pd.Series(python_words).value_counts()

# Calculate the word frequency for Java language
java_freq = pd.Series(java_words).value_counts()

# Calculate the word frequency for JavaScript language
script_freq = pd.Series(script_words).value_counts()

# Calculate the word frequency for HTML language
html_freq = pd.Series(html_words).value_counts()

# Calculate the word frequency for TypeScript language
type_freq = pd.Series(type_words).value_counts()

# Calculate the word frequency for all languages combined
all_freq = pd.Series(all_words).value_counts()

# Calculate the word frequency for words not categorized in specific languages
other_freq = pd.Series(other_words).value_counts()


**JAVASCRIPT**          

In [ ]:
# Display the top 10 most common words for the 'script' category in descending order
word_counts.script.sort_values(ascending=False).head(10)
# Top 3 words: id, desk, rel

In [ ]:
# Generate a word cloud image using the 'script_words' list, with a white background
img = WordCloud(background_color='white').generate(' '.join(script_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the plot
plt.title('Most Common JavaScript Words')

# Display the plot
plt.show()


In [ ]:
# Generate a word cloud image using the 'script_words' list, with a black background
colors = ["red", "yellow", "orange", "pink"]
img = WordCloud(background_color='black', colormap=colors).generate(' '.join(script_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the plot
plt.title('Most Common JavaScript Words')

# Display the plot
plt.show()


**PYTHON**            

In [ ]:
set_python_words = set(python_words)
set_python_words

In [ ]:
len(set_python_words)


In [ ]:
python_words

In [ ]:
# JavaScript, Other have the most words
len(python_words), len(java_words), len(script_words), len(html_words), len(type_words), len(other_words), len(all_words)


In [ ]:
# Retrieve the top 10 most frequent words in the 'python' category
word_counts.python.sort_values(ascending=False).head(10)

# Top 3 words: user, desk, rel 

In [ ]:
# Generate a word cloud visualization for the most common Python words
img = WordCloud(background_color='white').generate(' '.join(python_words))

# Display the generated image
plt.imshow(img)

# Turn off axis display
plt.axis('off')

# Set the title of the plot
plt.title('Most Common Python Words')

# Show the plot
plt.show()


**JAVA**                

In [ ]:
# Display the top 10 most common words in the Java programming language
word_counts.java.sort_values(ascending=False).head(10)

#Top 3 words: setting, id, version


In [ ]:
# Generate and display a word cloud visualization for the most common Java words
img = WordCloud(background_color='white').generate(' '.join(java_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the word cloud plot
plt.title('Most Common Java Words')

# Show the plot
plt.show()


In [ ]:
# Generate a word cloud image using the 'script_words' list, with a black background
colors = ["red", "yellow", "orange", "pink"]
img = WordCloud(background_color='black', colormap=colors).generate(' '.join(all_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the plot
plt.title('All Most Common Words')

# Display the plot
plt.show()


**HTML**                                 

In [ ]:
# Display the top 10 most common words in the HTML programming language
word_counts.html.sort_values(ascending=False).head(10)

# Top 3 words: body, id, reference


In [ ]:
# Generate a word cloud image using the 'HTML' list, with a black background
colors = ["red", "yellow", "orange", "pink"]
img = WordCloud(background_color='black', colormap=colors).generate(' '.join(all_words))


**TYPESCRIPT**           

In [ ]:
# Display the top 10 most common words in the TypeScript programming language
word_counts.type.sort_values(ascending=False).head(10)

# Top 3 words: mount, round, overview.However, there is a small sample size here, so this is likely not representative.

In [ ]:
# Generate a word cloud image using the 'script_words' list, with a black background
colors = ["red", "yellow", "orange", "pink"]
img = WordCloud(background_color='black', colormap=colors).generate(' '.join(type_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the plot
plt.title('Most Common TypeScript Words')
# Display the plot
plt.show()


**OTHER**       

In [ ]:
word_counts.other.sort_values(ascending=False).head(10)

# Top 3 words: height, setting, reference

In [ ]:
# Generate a word cloud image using the 'other' list, with a black background
colors = ["red", "yellow", "orange", "pink"]
img = WordCloud(background_color='black', colormap=colors).generate(' '.join(other_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the plot
plt.title('Most Common Other Programming Language Words')

# Display the plot
plt.show()


#### ALL LANGUAGE OBSERVED

In [ ]:
# Concatenate frequency DataFrames for each language and the overall frequency axis=1 concatenates the DataFrames horizontally (side by side)

# Fill missing values (NaNs) with 0 and convert all values to integers
word_counts = pd.concat([python_freq, java_freq, script_freq, html_freq, type_freq, other_freq, all_freq], axis=1).fillna(0).astype(int)

# Rename the column names to match the respective programming languages and an 'all' category
word_counts.columns = ['python', 'java', 'script', 'html', 'type', 'other', 'all']

# Display the first few rows of the 'word_counts' DataFrame
word_counts.head()


In [ ]:
# Display the top 10 most common words on ALL programming languages
word_counts['all'].sort_values(ascending=False).head(10)



In [ ]:
# Generate a word cloud image using the 'script_words' list, with a black background
colors = ["red", "yellow", "orange", "pink"]
img = WordCloud(background_color='black', colormap=colors).generate(' '.join(all_words))

# Display the word cloud image without axis
plt.imshow(img)
plt.axis('off')

# Set the title of the plot
plt.title('All Most Common Words')

# Display the plot
plt.show()


### Question 1: Are there any notable variations in the frequency of words between README files written in different programming languages?

In [ ]:
# Sort the DataFrame 'word_counts' in descending order based on the 'all' column. This will arrange the words in the DataFrame by their frequency in descending order

word_counts.sort_values('all', ascending=False)

In [ ]:
# Set the default figure size for matplotlib plots to (13, 7)
plt.rc('figure', figsize=(13, 7))

# Apply the 'seaborn-darkgrid' style to matplotlib plots.This style provides a dark background with gridlines, enhancing visualization
plt.style.use('seaborn-darkgrid')


In [ ]:
# Sort the 'word_counts' DataFrame based on the 'all' column in descending order,then select the top 20 rows and columns 'python', 'script', and 'other'.Finally, create a horizontal bar plot to visualize the top 20 words for each language
word_counts.sort_values('all', ascending=False)[['python', 'script', 'other']].head(20).plot.barh()


**The words in each language seem to vary in the same way. This may be an artifact of cleaning the words by only accepting dictionary words.**

### Question 2: Does the presence of specific libraries in the README file correspond with the programming language used?

In [ ]:
def analyze_word_frequency(train):
    """
    Analyzes word frequency in specific libraries/tools across different programming languages.

    Args:
    train (pandas.DataFrame): DataFrame containing the training data.

    Returns:
    word_counts (pandas.DataFrame): DataFrame with word counts for each programming language, overall, and specific libraries/tools.

    """
    # Get words for each programming language and overall
    categories = ['JavaScript', 'Python', 'Java', 'HTML','TypeScript','Other']
    all_words = clean(' '.join(train['clean_contents']))
    category_words = [clean(' '.join(train[train.language == category]['clean_contents'])) for category in categories]

    # Get word frequency for each category
    category_words_freq = [pd.Series(words).value_counts() for words in category_words]
    
    # Filter words by specific library
    libraries_to_check = ['flexbox', 'chatgpt', 'cli', 'stackblitz', 'angular', 'apm', 'opencv', 'zendesk', 'bootstrap', 'jquery', 'virtualbox', 'vagrant', 'nbsp', 'machinelearning', 'apache', 'dubbo', 'alibaba', 'pandas', 'numpy']
    filtered_list = [word for word in all_words if word in libraries_to_check]
    filtered_list_freq = pd.Series(filtered_list).value_counts()

    # Create DataFrame with word counts
    word_counts = pd.concat(category_words_freq + [pd.Series(filtered_list_freq)], axis=1).fillna(0).astype(int)

    # Rename columns
    word_counts.columns = categories + ['All', 'Tools_Frameworks']

    # Visualize
    word_counts.sort_values('Tools_Frameworks', ascending=False)[categories].head(14).plot.barh()
    plt.xlabel('Word Count')
    plt.title('Word Count by Language and Specific Library')
    plt.show()

    return word_counts

# Call the function
word_counts_result = analyze_word_frequency(train)
word_counts_result.head()


### Question 3: What are the most frequently used words throughout the dataset and for each language?

In [ ]:
# Access the 'all' column from the 'word_counts' DataFrameThen, sort the values in descending order to get the most frequent words first

# Use .head(10) to extract the top 10 most frequent words
most_common_words = word_counts['all'].sort_values(ascending=False).head(10)


In [ ]:
# Generate a WordCloud image from the joined 'all_words' list

# The WordCloud will have a white background
img = WordCloud(background_color='white').generate(' '.join(all_words))

# Display the WordCloud image using plt.imshow()
plt.imshow(img)

# Turn off the axis to remove axes labels and ticks
plt.axis('off')

# Set the title for the WordCloud visualization
plt.title('All Most Common Words')

# Display the WordCloud visualization
plt.show()


Top 3 words: height, must, reference

Words that seem to be common among coding languages: title, desk, height, blob# Concatenate frequency DataFrames for each language and the overall frequency axis=1 concatenates the DataFrames horizontally (side by side)

### Question 4: What are the least frequently used words throughout the dataset and for each language?

In [ ]:
def least_used_words_per_language(train):
    """
    Determines the least used words and their corresponding language across different programming languages.

    Args:
    train (pandas.DataFrame): DataFrame containing the training data.

    Returns:
    least_used_words_per_column (pandas.Series): Series containing the least used words and their corresponding language.

    """
    # Gather words for each programming language and overall
    categories = ['JavaScript', 'Python', 'Java', 'HTML','TypeScript','Other']
    all_words = clean(' '.join(train['clean_contents']))
    category_words = [clean(' '.join(train[train.language == category]['clean_contents'])) for category in categories]

    # Calculate word frequency for each category
    category_words_freq = [pd.Series(words).value_counts() for words in category_words]

    # Combine word frequencies for analysis
    word_counts = pd.concat(category_words_freq, axis=1).fillna(0).astype(int)

    # Rename columns for clarity
    word_counts.columns = categories

    # Sort columns based on overall word count
    word_counts_sorted = word_counts.sort_values('All', ascending=False)

    # Calculate the total count of words across all columns
    word_counts_sorted['Total'] = word_counts_sorted.sum(axis=1)

    # Extract the least used words per programming language
    least_used_words_per_column = word_counts_sorted.idxmin()

    return least_used_words_per_column

# Call the function and display the result
result_least_used_words = least_used_words_per_language(train)
print(result_least_used_words)


### Feature will be tested and modeled against Attrition

In [ ]:
best_model_classification_matrix(X_bow, y_train, X_validate_bow, y_validate)


In [ ]:
#  # Visualize the data

# # Example: Histogram of 'Age'
# plt.figure(figsize=(8, 6))
# sns.histplot(selected_train['Age'], kde=True)
# plt.title('Age Distribution')
# plt.xlabel('Age')
# plt.ylabel('Count')
# plt.show()

In [ ]:
# #  Explore the Age vs Attrition

# # Example: Bar plot of 'Age' vs. 'Attrition'
# plt.figure(figsize=(10, 6))
# sns.countplot(data=selected_train, x='Age', hue='Yes_Attrition')
# plt.title('Does Age have a relationship with employee Attrition?')
# plt.xlabel('Age')
# plt.ylabel('Count')
# plt.show()

In [ ]:
# # Example: Bar plot of 'Education' vs. 'Attrition''

# plt.figure(figsize=(10, 8))
# sns.countplot(data=selected_train, x='Education', hue='Yes_Attrition')
# plt.title('Does Education have a relationship with employee Attrition?')
# plt.xlabel('Education')
# plt.ylabel('Count')
# plt.show()

In [ ]:
# plt.figure(figsize=(8, 6))
# sns.kdeplot(data=selected_df, x='Tenure', hue='Yes_Attrition', fill=True)
# plt.title('Does Tenure have a relationship with employee Attrition?')
# plt.xlabel('Tenure')
# plt.ylabel('Density')
# plt.show()

# STATISTICAL TESTING

### Question 1: Are there any notable variations in the frequency of words between README files written in different programming languages?

- **$H_{0}$** There is no significant variation in the frequency of words between README files written in different programming languages.
- **$H_{a}$** There are significant variations in the frequency of words between README files written in different programming languages.

### Question 2: Does the presence of specific libraries in the README file correspond with the programming language used?

- **$H_{0}$** The presence of specific libraries in the README file does not correspond with the programming language used.
- **$H_{a}$** The presence of specific libraries in the README file corresponds with the programming language used.

### Question 3: What are the most frequently used words throughout the dataset and for each language?

- **$H_{0}$** The frequency distribution of words is similar throughout the dataset and across different programming languages.
- **$H_{a}$** The frequency distribution of words is significantly different throughout the dataset and across different programming languages.

### Question 4: What are the least frequently used words throughout the dataset and for each language?
- **$H_{0}$** The least frequent words have consistent usage patterns throughout the dataset and across different programming languages.
- **$H_{a}$** The least frequent words have varying usage patterns throughout the dataset and across different programming languages.

# STATISTICAL FINDINGS
- ge Hypothesis - We reject the Null Hypothesis, Age is dependent on employee Attrition.
- Education Hypothesis We reject the Null Hypothesis, education is dependent on on employee Attrition.
- Tenure Hypothesis - We fail to reject the Null Hypothesis, Tenure is independent of employee Attrition.
- 

# SPLITTING DATA

In [ ]:
train, validate, test = m.split_data(words_df, 'language')


In [ ]:
train


In [ ]:
#Split the data in to X and Y for all datasets with Yes_Attrition and those without 

X_bow, X_validate_bow, X_test_bow, y_train, y_validate, y_test, feature_names = prepare_for_modeling(train, validate, test)

In [ ]:
scores_df = decision_tree(X_bow, X_validate_bow, y_train, y_validate)
scores_df

In [ ]:
selected_y_train.head()
selected_y_train.shape

### EVALUATE BASELINE

In [ ]:
# baseline =(selected_y_train==0).mean()
# print(f'The baseline accuracy is: {baseline:.2%}')

# MODELING

### DECISION TREE

### RANDOM FOREST

### KNN

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

def k_nearest(X_bow, y_train, X_validate_bow, y_validate):
    """
    Trains and evaluates KNN models for different values of k and plots the results.

    Parameters:
    -----------
    X_bow: array-like, shape (n_samples, n_features)
        Bag-of-words representations of training samples.
    y_train: array-like, shape (n_samples,)
        Target values for the training samples.
    X_validate_bow: array-like, shape (n_samples, n_features)
        Bag-of-words representations of validation samples.
    y_validate: array-like, shape (n_samples,)
        Target values for the validation samples.

    Returns:
    --------
    results: pandas DataFrame
        Contains the train and validation accuracy for each value of k.
    """
    # KNN model evaluation for different values of k
    metrics = []
    train_score = []
    validate_score = []
    for k in range(1, 21):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_bow, y_train)
        train_score.append(knn.score(X_bow, y_train))
        validate_score.append(knn.score(X_validate_bow, y_validate))
        diff_score = train_score[-1] - validate_score[-1]
        metrics.append({'k': k, 'train_score': train_score[-1], 'validate_score': validate_score[-1], 'diff_score': diff_score})

    baseline_accuracy = (y_train == 6).mean()

    results = pd.DataFrame.from_records(metrics)

    # Plot results
    # (visualization code here)

    return results





### Question 2: Does the presence of specific libraries in the README file correspond with the programming language used?

### Question 4: What are the least frequently used words throughout the dataset and for each language?


### TRAIN SPLIT

# EXPLORE

In [ ]:
cleaned_list


In [ ]:
se = pd.Series(cleaned_list)
words_df['cleaned_text'] = se.values


In [ ]:
words_df


In [ ]:
train, validate, test = split_data(df, 'language')
train.head()

In [ ]:
    separate specific words by language category

In [ ]:
# we can do that process with a join on a Series and not just a list
# we will do that all words and categories
# we will pass our basic cleaning on top of that

JavaScript_words = clean(' '.join(train[train.language=='JavaScript']['clean_contents']))
Python_words = clean(' '.join(train[train.language=='Python']['clean_contents']))
Java_words = clean(' '.join(train[train.language=='Java']['clean_contents']))
TypeScript_words = clean(' '.join(train[train.language=='TypeScript']['clean_contents']))
HTML_words = clean(' '.join(train[train.language=='HTML']['clean_contents']))
Other_words = clean(' '.join(train[train.language=='Other']['clean_contents']))
all_words = clean(' '.join(train['clean_contents']))